In [1]:
from datasets import load_dataset

dataset = load_dataset("nid989/FNC-1")

c:\Users\ryguy\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset csv (C:/Users/ryguy/.cache/huggingface/datasets/nid989___csv/nid989--FNC-1-ba6c09ed0c9efe30/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d)
100%|██████████| 3/3 [00:00<00:00, 12.92it/s]


In [2]:
train = dataset['train']
test = dataset['test']
val = dataset['validation']

In [3]:
train[3]

{'Headline': 'Kimye’s Fault? Anna Wintour’s NYC ‘Vogue’ Offices Hit With Disgusting Rat Infestation: ‘Rat Droppings On Desks’',
 'articleBody': 'YouTube prankster Josh Paler Lin decided give man named Thomas money follow see spends it. His first stop local liquor store instead buying alcohol purchased food gave away others need. A homeless man given chance, touched awful lot surprised people. A viral online video YouTube prankster Josh Paler Lin tells story homeless man named Thomas desire let circumstances keep selfless - using newfound money help himself, others. Josh made video, received 11 million hits Wednesday morning, idea give random homeless person $100 secretly follow see spends it. He meets elderly man gray hair, mustache dark jacket named Thomas holding sign along highway. He tells Josh he\'s trying get enough buy something eat Josh shocks $100. "Oh goodness. Oh way," says Josh pulls five $20 bills wallet. "Oh brother, sure? I\'m starting tear here. That\'s like incredible.

In [4]:
#https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
import re
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won\'t", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [5]:
# Remove puncuation 
# NOTE: HANDLE Sentence Level
import re
import numpy as np
def cleanText(textData):
    # Lowercase
    textDataLower = np.array(list(map(lambda x: x.lower(),textData)))
    # Expand contradictions
    textNoContras = np.array(list(map(lambda x: decontracted(x),textDataLower)))
    # Remove Puncuation
    textDataNoPunc = np.array(list(map(lambda x: re.sub(r'\.|,|\(|\)|\?|!|\n|;|:','',x), textNoContras)))
    # Remove numbers
    textNoNumbers = np.array(list(map(lambda x: re.sub(r'(0|1|2|3|4|5|6|7|8|9|0)+','',x),textDataNoPunc)))
    # Remove extra spaces
    cleanedText = np.array(list(map(lambda x: re.sub(r' +',' ',x),textNoNumbers)))
    return np.array(list(map(lambda x: x.split(' '),cleanedText)))

In [6]:
trainHeadlines = cleanText(train['Headline'])
trainBodies = cleanText(train['articleBody'])
valHeadlines = cleanText(val['Headline'])
valBodies = cleanText(val['articleBody'])
testHeadlines = cleanText(test['Headline'])
testBodies = cleanText(test['articleBody'])
yTrain = train['Stance']
yVal = val['Stance']
yTest = test['Stance']

C:\Users\ryguy\AppData\Local\Temp\ipykernel_4496\3918129584.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(list(map(lambda x: x.split(' '),cleanedText)))


In [14]:
import gensim.downloader as api
wv = api.load('glove-twitter-50')

[==================================================] 100.0% 199.5/199.5MB downloaded


In [15]:
vectorSize = 50
def vectorizeWord(word):
    if (word in wv): return wv[word][0:vectorSize]
    return [0]*vectorSize

In [16]:
def vectorizeSent(sent):
    return np.array(list(map(lambda x: vectorizeWord(x),sent)))

In [17]:
vectorTrainHeadlines = np.array(list(map(lambda x: vectorizeSent(x),trainHeadlines)))
vectorTrainBodies = np.array(list(map(lambda x: vectorizeSent(x),trainBodies)))

vectorValHeadlines = np.array(list(map(lambda x: vectorizeSent(x),valHeadlines)))
vectorValBodies = np.array(list(map(lambda x: vectorizeSent(x),valBodies)))

vectorTestHeadlines = np.array(list(map(lambda x: vectorizeSent(x),testHeadlines)))
vectorTestBodies = np.array(list(map(lambda x: vectorizeSent(x),testBodies)))

C:\Users\ryguy\AppData\Local\Temp\ipykernel_4496\3930145267.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  vectorTrainHeadlines = np.array(list(map(lambda x: vectorizeSent(x),trainHeadlines)))
C:\Users\ryguy\AppData\Local\Temp\ipykernel_4496\3930145267.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  vectorTrainBodies = np.array(list(map(lambda x: vectorizeSent(x),trainBodies)))
C:\Users\ryguy\AppData\Local\Temp\ipykernel_4496\3930145267.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tupl

In [18]:
# Find Average Vector of all entries
def getAverageVector(vectors):
    return np.average(vectors,axis=0)

In [19]:
averageVectorTrainHeadlines = np.array(list(map(lambda x: getAverageVector(x),vectorTrainHeadlines)))
averageVectorBodyTrain = np.array(list(map(lambda x: getAverageVector(x),vectorTrainBodies)))

averageVectorValHeadlines = np.array(list(map(lambda x: getAverageVector(x),vectorValHeadlines)))
averageVectorValBodies = np.array(list(map(lambda x: getAverageVector(x),vectorValBodies)))

averageVectorTestHeadlines = np.array(list(map(lambda x: getAverageVector(x),vectorTestHeadlines)))
averageVectorTestBodies = np.array(list(map(lambda x: getAverageVector(x),vectorTestBodies)))

In [20]:
# Headlines = 10
# Body = 90
def pad(arr,size):
    if (len(arr)>size): return arr[0:size]
    elif len(arr)<size: return np.append(arr, np.array((size-len(arr))*[[0]*50]),axis=0)
    return arr

In [21]:
padTrainHeadlines = np.array(list(map(lambda x: pad(x,10),vectorTrainHeadlines)))
padValHeadlines = np.array(list(map(lambda x: pad(x,10),vectorValHeadlines)))
padTestHeadlines = np.array(list(map(lambda x: pad(x,10),vectorTestHeadlines)))

padTrainBodies = np.array(list(map(lambda x: pad(x,90),vectorTrainBodies)))
padValBodies = np.array(list(map(lambda x: pad(x,90),vectorValBodies)))
padTestBodies = np.array(list(map(lambda x: pad(x,90),vectorTestBodies)))

In [22]:
concatTrain = []
for i in range(len(vectorTrainHeadlines)):
    concatTrain.append(np.concatenate((padTrainHeadlines[i],[averageVectorTrainHeadlines[i]+averageVectorBodyTrain[i]],padTrainBodies[i])))
concatTrain = np.array(concatTrain)

In [ ]:
concatTrain.shape

(40476, 101, 50)

(40476, 101, 50)

In [24]:
concatVal = []
for i in range(len(vectorValHeadlines)):
    concatVal.append(np.concatenate((padValHeadlines[i],[averageVectorValHeadlines[i]+averageVectorValBodies[i]],padValBodies[i])))
concatVal = np.array(concatVal)

In [25]:
concatVal.shape

(4498, 101, 50)

In [26]:
concatTest = []
for i in range(len(vectorTestHeadlines)):
    concatTest.append(np.concatenate((padTestHeadlines[i],[averageVectorTestHeadlines[i]+averageVectorTestBodies[i]],padTestBodies[i])))
concatTest = np.array(concatTest)

In [27]:
concatTest.shape

(4998, 101, 50)

In [28]:
def labelsToBinary(labels):
    return np.array(list(map(lambda x: 0 if x==3 else 1,labels)))

In [29]:
yTrainBinary = labelsToBinary(yTrain)
yValBinary = labelsToBinary(yVal)
yTestBinary = labelsToBinary(yTest)

In [30]:
counts = [0,0,0,0]
for i in yTrainBinary:
    counts[i]+=1
counts

[29544, 10932, 0, 0]

In [31]:
import tensorflow as tf
from tensorflow.keras import layers
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(101,50),dtype=tf.float32))

model.add(layers.Conv1D(32,3))
model.add(layers.MaxPooling1D())

model.add(layers.Conv1D(48,3))
model.add(layers.MaxPooling1D())

model.add(layers.Bidirectional(layers.LSTM(64)))
model.add(layers.Dropout(.2))

model.add(layers.Dense(256))
model.add(layers.Dense(256))
model.add(layers.Dense(2,activation="softmax"))
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [32]:
model.fit(np.array(list(concatTrain)),np.array(list(yTrainBinary)),
          validation_data=(np.array(list(concatVal)),np.array(list(yValBinary))),epochs=15)

Epoch 1/15
1265/1265 [==============================] - 45s 29ms/step - loss: 0.4404 - accuracy: 0.7883 - val_loss: 0.2940 - val_accuracy: 0.8626
Epoch 2/15
1265/1265 [==============================] - 35s 27ms/step - loss: 0.2531 - accuracy: 0.8887 - val_loss: 0.2054 - val_accuracy: 0.9117
Epoch 3/15
1265/1265 [==============================] - 32s 25ms/step - loss: 0.1794 - accuracy: 0.9234 - val_loss: 0.1730 - val_accuracy: 0.9291
Epoch 4/15
1265/1265 [==============================] - 32s 25ms/step - loss: 0.1423 - accuracy: 0.9417 - val_loss: 0.1609 - val_accuracy: 0.9293
Epoch 5/15
1265/1265 [==============================] - 30s 24ms/step - loss: 0.1147 - accuracy: 0.9533 - val_loss: 0.1515 - val_accuracy: 0.9422
Epoch 6/15
1265/1265 [==============================] - 31s 24ms/step - loss: 0.1002 - accuracy: 0.9600 - val_loss: 0.1354 - val_accuracy: 0.9453
Epoch 7/15
1265/1265 [==============================] - 30s 24ms/step - loss: 0.0837 - accuracy: 0.9676 - val_loss: 0.1272 -

In [33]:
preds = model.predict(concatTest)

157/157 [==============================] - 3s 10ms/step


In [34]:
predsActual = np.array(list(map(lambda x: np.argmax(x),preds)))
predsActual

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [35]:
from sklearn.metrics import accuracy_score
accuracy_score(yTestBinary,predsActual)

0.9635854341736695

In [36]:
# Get data for training the second order model
def getSecondOrderData(x,y):
    newX = []
    newY = []
    for i in range(len(y)):
        if (y[i]!=3):
            newX.append(x[i])
            newY.append(y[i])
    return (np.array(newX),np.array(newY))

In [37]:
xTrainTwo, yTrainTwo = getSecondOrderData(concatTrain,yTrain)
xValTwo, yValTwo = getSecondOrderData(concatVal, yVal)
xTestTwo, yTestTwo = getSecondOrderData(concatTest, yTest)

In [38]:
modelTwo = tf.keras.Sequential()
modelTwo.add(tf.keras.Input(shape=(101,50),dtype=tf.float32))

modelTwo.add(layers.Conv1D(32,3))
modelTwo.add(layers.MaxPooling1D())

modelTwo.add(layers.Conv1D(48,3))
modelTwo.add(layers.MaxPooling1D())

modelTwo.add(layers.Bidirectional(layers.LSTM(64)))
modelTwo.add(layers.Dropout(.2))

modelTwo.add(layers.Dense(256))
modelTwo.add(layers.Dense(256))
modelTwo.add(layers.Dense(3,activation="softmax"))
modelTwo.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [39]:
modelTwo.fit(np.array(list(xTrainTwo)),np.array(list(yTrainTwo)),
          validation_data=(np.array(list(xValTwo)),np.array(list(yValTwo))),epochs=13)

Epoch 1/13
342/342 [==============================] - 14s 23ms/step - loss: 0.6934 - accuracy: 0.7055 - val_loss: 0.6322 - val_accuracy: 0.7166
Epoch 2/13
342/342 [==============================] - 7s 20ms/step - loss: 0.5032 - accuracy: 0.7927 - val_loss: 0.4522 - val_accuracy: 0.8119
Epoch 3/13
342/342 [==============================] - 7s 21ms/step - loss: 0.3577 - accuracy: 0.8618 - val_loss: 0.3761 - val_accuracy: 0.8512
Epoch 4/13
342/342 [==============================] - 7s 21ms/step - loss: 0.2735 - accuracy: 0.8925 - val_loss: 0.2979 - val_accuracy: 0.8888
Epoch 5/13
342/342 [==============================] - 7s 21ms/step - loss: 0.2083 - accuracy: 0.9188 - val_loss: 0.2708 - val_accuracy: 0.8997
Epoch 6/13
342/342 [==============================] - 7s 21ms/step - loss: 0.1760 - accuracy: 0.9333 - val_loss: 0.2758 - val_accuracy: 0.9038
Epoch 7/13
342/342 [==============================] - 6s 19ms/step - loss: 0.1463 - accuracy: 0.9417 - val_loss: 0.3088 - val_accuracy: 0.904

In [40]:
# Predictions
# Get First order predictions
finalPreds = []
orderOnePreds = np.array(list(map(lambda x: np.argmax(x),model.predict(concatTest))))
orderTwoPreds = np.array(list(map(lambda x: np.argmax(x),modelTwo.predict(concatTest))))
for i in range(len(orderOnePreds)):
    if (orderOnePreds[i]==0):
        finalPreds.append(3)
    else:
        finalPreds.append(orderTwoPreds[i])

157/157 [==============================] - 2s 7ms/step


In [41]:
counts = [0,0,0,0]
for i in finalPreds:
    counts[i]+=1
counts

[362, 57, 928, 3651]

In [42]:
from sklearn.metrics import accuracy_score
accuracy_score(finalPreds,yTest)

0.951780712284914